### Install Dependencies for Accessing and Transformation Procedures

In [1]:
!pip3 install pandas
!pip3 install s3fs
!pip3 install boto3
!pip3 install kagglehub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 85.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2025.2.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda

In [2]:
import os
import boto3
import s3fs
import pandas as pd

from datetime import datetime

#### Download Dataset from Kaggle  
This cell uses the `kagglehub` library to download the latest version of the Twitter dataset from Kaggle. The dataset is stored in the specified path, which is printed for reference.  


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mmmarchetti/tweets-dataset")

print("Path to dataset files:", path)

100%|██████████| 2.82M/2.82M [00:00<00:00, 46.9MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/mmmarchetti/tweets-dataset/versions/1


#### Locate the CSV File  
After downloading the dataset, this cell identifies the path to the `tweets.csv` file within the downloaded directory. The path is printed for verification.

In [4]:
# Find CSV file in the downloaded directory
twitter_csv_path = os.path.join(path, "tweets.csv")

print("Dataset downloaded to:", twitter_csv_path)

Dataset downloaded to: /root/.cache/kagglehub/datasets/mmmarchetti/tweets-dataset/versions/1/tweets.csv


#### Load and Inspect the Dataset  
The downloaded `tweets.csv` file into a Pandas DataFrame. The `delimiter` is set to `,` and the encoding to `utf-8` to ensure proper parsing. The first few rows of the dataset are displayed using `df.head()`, and the column names and shape of the DataFrame are printed for inspection.  

In [5]:
# Load the dataset into a Pandas DataFrame
df = pd.read_csv(twitter_csv_path, delimiter = ',', encoding = 'utf-8')

# Display the first 5 rows of the DataFrame
df.head()

,author,content,country,date_time,id,language,latitude,longitude,number_of_likes,number_of_shares
0,katyperry,Is history repeating itself...?#DONTNORMALIZEH...,NaN,12/01/2017 19:52,8.196330e+17,en,NaN,NaN,7900,3472
1,katyperry,@barackobama Thank you for your incredible gra...,NaN,11/01/2017 08:38,8.191010e+17,en,NaN,NaN,3689,1380
2,katyperry,Life goals. https://t.co/XIn1qKMKQl,NaN,11/01/2017 02:52,8.190140e+17,en,NaN,NaN,10341,2387
3,katyperry,Me right now 🙏🏻 https://t.co/gW55C1wrwd,NaN,11/01/2017 02:44,8.190120e+17,en,NaN,NaN,10774,2458
4,katyperry,SISTERS ARE DOIN' IT FOR THEMSELVES! 🙌🏻💪🏻❤️ ht...,NaN,10/01/2017 05:22,8.186890e+17,en,NaN,NaN,17620,4655


In [6]:
# Print column names
df.columns

Index(['author', 'content', 'country', 'date_time', 'id', 'language',
       'latitude', 'longitude', 'number_of_likes', 'number_of_shares'],
      dtype='object')

In [7]:
# Print the shape of the DataFrame (rows, columns)
df.shape

(52542, 10)

#### Configure AWS Credentials  
The AWS credentials (access key, secret key, and region) to enable interaction with AWS services like S3, Glue and Redshift.

In [8]:
AWS_ACCESS_KEY = ''
AWS_SECRET_KEY = ''
AWS_REGION = 'us-east-2'

In [9]:
try:
  session = boto3.Session(
      aws_access_key_id = AWS_ACCESS_KEY,
      aws_secret_access_key = AWS_SECRET_KEY,
      region_name = AWS_REGION
  )

  print("Session Created Successfully")

except Exception as e:
  print(e)

Session Created Successfully


#### Create S3 Bucket for Raw Data  
This cell create an S3 bucket named `twitter-raw-dev` in the `us-east-2` region to store the raw Twitter dataset.

In [10]:
try:
  aws_s3 = session.client('s3')

  s3_bucket_name = 'twitter-raw-dev'
  file_name = twitter_csv_path

  aws_s3.create_bucket(Bucket = s3_bucket_name,
                       CreateBucketConfiguration={
        'LocationConstraint': 'us-east-2' }
  )

  print(f"Created S3 Bucket {s3_bucket_name}")

except Exception as e:
  print(e)

An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


#### Inspect Data Types and Missing Values  
This cell checks the data types of each column in the DataFrame using `df.dtypes` and identifies missing values using `df.isnull().sum()`. This step is crucial for understanding the structure and quality of the dataset before proceeding with transformations.  

In [11]:
print(df.dtypes)
print(df.isnull().sum())  # Check missing values

author               object
content              object
country              object
date_time            object
id                  float64
language             object
latitude            float64
longitude           float64
number_of_likes       int64
number_of_shares      int64
dtype: object
author                  0
content                 0
country             52506
date_time               0
id                      0
language                0
latitude            52541
longitude           52541
number_of_likes         0
number_of_shares        0
dtype: int64


#### Rename Columns and Save as Parquet  
This renames the columns of the DataFrame for better readability and consistency. After renaming, the processed dataset is saved in Parquet format
`processed_data.parquet`

In [12]:
df.columns = ["author", "content", "country", "date_time", "id", "language",
              "latitude", "longitude", "number_of_likes", "number_of_shares"]

df.to_parquet("processed_data.parquet", index=False)  # Save as Parquet (recommended)

#### Upload Processed Data to S3  
This cell uploads the processed Parquet file (`processed_data.parquet`) to the `twitter-raw-dev` S3 bucket under the `raw/` directory.

In [13]:
try:
  aws_s3.upload_file('processed_data.parquet', s3_bucket_name, "raw/tweets.parquet")

  print("File Uploaded Successfully")

except Exception as e:
  print(e)

File Uploaded Successfully


In [14]:
import psycopg2

try:
    conn = psycopg2.connect(
        dbname='',
        user='',
        password='',
        host='',  # Cluster endpoint
        port='5439'
    )
    print("Connection successful!")
    conn.close()
except Exception as e:
    print(f"Connection failed: {e}")

Connection successful!
